### Video Indexer API Samples (Python)

This notebooks provides samples for the following operations in Video Indexer:   
(1) Get account detailes   
(2) Upload a video from URL   
(2A) Upload a video from local file   
(3) Wait for the video to finish indexing   
(4) Search for video and get insights   
(5) Widgets API <br>
(6) Get Video Insights <br>
(7) Get Video transcription translation <br>
(8) Store JSON payload into container   
   
Make sure you're logged-in with `az` to authenticate your account.   
   
Copy the `.env.example` file to a new file named `.env`, and update the values with your own account settings.

In [1]:
# install python dotenv
%pip install python-dotenv

# install Azure blob storage to upload the json file into storage container
%pip install azure-storage-blob

In [ ]:
# login to Azure to access Video indexor portal
!az login

In [1]:
# imports custom VI client
from dotenv import dotenv_values
from VideoIndexer.Consts import Consts
from VideoIndexer.VideoIndexerClient import VideoIndexerClient

Define the following parameters based on your account settings, in case they are different or not defined as environment variables:

In [2]:
# load AI configuration from. env
config = dotenv_values(".env")
AccountName = config.get('AccountName')
ResourceGroup = config.get('ResourceGroup')
SubscriptionId = config.get('SubscriptionId')

# print(AccountName)
# print(ResourceGroup)
# print(SubscriptionId)

ApiVersion = '2024-01-01'
ApiEndpoint = 'https://api.videoindexer.ai'
AzureResourceManager = 'https://management.azure.com'

# create and validate consts
consts = Consts(ApiVersion, ApiEndpoint, AzureResourceManager, AccountName, ResourceGroup, SubscriptionId)


# create Video Indexer Client
client = VideoIndexerClient()


# Get access tokens (arm and Video Indexer account)
client.authenticate_async(consts)

#### Sample 1 - Get Account Basic Details
Get account details, not required in most cases.

In [3]:
client.get_account_async()

[Account Details] Id:3db49057-43b0-4a5e-9829-c4c8c9651170, Location: uksouth


#### Sample 2 - Index a Video from URL
Upload a video from URL, do not wait for the index operation to complete.   
Define `videoUrl` and `ExcludedAI` first.   
`ExcludedAI` - A list of the AIs you would like to exclude in the format `['Faces', 'Labels', 'Emotions','ObservedPeople']`. Leave empty if you do not want to exclude any AIs. For more details see [here](https://api-portal.videoindexer.ai/api-details#api=Operations&operation=Upload-Video:~:text=AI%20to%20exclude%20when%20indexing%2C%20for%20example%20for%20sensitive%20scenarios.%20Options%20are%3A%20Face/Observed%20peopleEmotions/Labels%7D).

In [ ]:
# Give sample Video file for indexing from blob storage or from any url
# VideoUrl = 'https://mediaservicestr.blob.core.windows.net/sample-videos/Ukraine War_ Dramatic drone footage shows Russian convoy ambush-no audio.mp4'
VideoUrl = 'xxxxxxx'
ExcludedAI = []

# VI detect the source language. Possible value 'auto', 'multi'
# 'auto' - use when only one main langauage used in the video
# 'multi' - use when only multiple langauage used in the video
# source_lang = 'auto'
source_lang = 'multi-language'
video_id = client.upload_url_async('videoplayback-Mutli_Lang2', VideoUrl, ExcludedAI, False,source_lang)

#### Sample 2A - Index a Video From File
Upload From Local File.   
Define `LocalVideoPath` first.

In [ ]:
# sample video file from local path
LocalVideoPath = 'YOUR_LOCAL_VIDEO_FILE_PATH'

file_video_id = client.file_upload_async(LocalVideoPath, 'my-other-video-name', ExcludedAI)

#### Sample 3 - Polling on Video Completion Event
Wait for the video index to finish (Polling method).

In [ ]:
# client.wait_for_index_async(file_video_id)

# If you are using AI VI portal to upload sample video file copy video id from portal and config video_id name here
# video_id ="cec56c9fbc"
client.wait_for_index_async(video_id)

#### Sample 4 - Search for Video and Get insights
Search for the video and get insights.

In [ ]:
# get the VI response
resp = client.get_video_async(video_id)
print("resp****:",resp)

#### Sample 5 - Widgets API

In [ ]:
# from IPython.display import display, HTML

# client.get_insights_widgets_url_async(video_id, widget_type='Keywords')
# client.get_player_widget_url_async(video_id)

#response = client.get_video_insights_widgets_url_async(video_id, widget_type='People,Sentiments,Keywords,Transcript,NamedEntities,DetectedObjects')
# response = client.get_video_insights_widgets_url_async(video_id, widget_type='People,Sentiments,Keywords')
# print("respone ---->",response)

# display(HTML(str(response)))

#client.get_insights_widgets_url_async('0645b0860f', widget_type='Keywords')
#client.get_player_widget_url_async('0645b0860f')

#### Sample 6 - Get Video Insights
Get insights such as Caption, Transcription, Auto detect lang, Translation (from foreign lang -> English) ,Named entities ,Object identification <br>
Store the insights as file into Azure Blob storage


In [ ]:
# from IPython.display import display, HTML

# get the VI transcription as JSON
responseJSON = client.get_video_insights(video_id)
print("respone ---->",responseJSON)

#### Sample 7 - Get Video transcription translation
Get video's transcription as translated to target langauge


In [ ]:
# get the VI transcription as translated to target language in JSON 
target_lang = 'en-GB'
responseJSON = client.get_video_transcription_translation(video_id,target_lang)
print("respone ---->",responseJSON)

#### Sample 8 - Store Video Insights
Store the extracted insights as file into Azure Blob storage


In [21]:
# Write JSON response into blob storage
# from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import json
filename = video_id+'-VIInsigtResponse.json' 
with open(filename, 'w') as f:
    json.dump(responseJSON, f)


In [ ]:

# upload the response JSON file into blob storage

import os
import time

from azure.identity import DefaultAzureCredential
from azure.storage.blob.aio import BlobServiceClient, BlobClient, ContainerClient

import asyncio

from azure.identity.aio import DefaultAzureCredential
from azure.storage.blob.aio import BlobServiceClient, BlobClient, ContainerClient


from azure.storage.blob import BlobClient

connection_str ="xxxxxxx"
container_name = "vi-response"
blob_name = f.name

blob = BlobClient.from_connection_string(conn_str=connection_str, container_name=container_name, blob_name=blob_name)

with open(filename, "rb") as data:
    blob.upload_blob(data)